# Kaggle OCRデータセットの前処理

このNotebookでは、Kaggle OCRデータセットからYOLO形式のラベルファイルを生成します。

## 処理の流れ
1. Google Driveのマウント
2. Gitリポジトリのクローン/更新
3. 環境変数の設定
4. 前処理スクリプトの実行（`.py`を呼ぶだけ）
5. 結果の確認（必要最低限）

## 使用方法について

このNotebookは「実行スクリプト」として機能します。

- **ロジックは全て`.py`に実装済み**（`preprocess/kaggle_labels.py`）
- **Colabでは既存のipynbを実行するだけ**
- **編集はCursor（ローカル）で行う**

### 前提条件
- 生データはGoogle Driveに配置済み
- リポジトリはGitHubで管理

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd /content
!git clone -b feature/onnx_recognizer https://github.com/masararutaru/last_assignment_progzissen.git
# %cd last_assignment_progzissen
# !git pull  # 毎回実行

# 環境変数の設定（パスを直書きしない）
# DATA_ROOT: データのルートディレクトリ（その下に raw/aidapearson_ocr などがある）
# OUT_ROOT: 出力のルートディレクトリ（その下に processed などがある）

#実際の処理したいデータは/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment/raw/aidapearson_ocrの下にbatch1~batch9とextraのディレクトリがある.さらにその中にJSONフォルダとbackground_imagesフォルダがある.
#処理した後のデータは/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment/processedの下に生成される(状況に応じて柔軟な拡張がかのうな設計に).

import os
os.environ["DATA_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"
os.environ["OUT_ROOT"] = "/content/drive/MyDrive/大学　講義/2年/後期/java_zissen/datasets/last_assignment"

print("環境変数設定完了")
print(f"DATA_ROOT: {os.environ.get('DATA_ROOT')}")
print(f"OUT_ROOT: {os.environ.get('OUT_ROOT')}")
print("\n説明:")
print("  DATA_ROOT: プロジェクトルート（その下に raw/ ディレクトリがある）")
print("  OUT_ROOT: 出力ルート（その下に processed/ ディレクトリがある）")

In [ ]:
# 前処理スクリプトの実行（.pyを呼ぶだけ）
import sys
from pathlib import Path

# パスを追加（リポジトリのpython/dataディレクトリ）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

# 前処理スクリプトをインポート
from preprocess.kaggle_labels import main

# 環境変数からパスを取得（プロジェクトルート）
data_root = os.environ.get("DATA_ROOT", "/content/drive/MyDrive/data")
out_root = os.environ.get("OUT_ROOT", "/content/drive/MyDrive/outputs")

# 引数を設定して実行
# DATA_ROOT/raw/aidapearson_ocr がソース
# OUT_ROOT/processed が出力先
source = str(Path(data_root) / "raw" / "aidapearson_ocr")
target = str(Path(out_root) / "processed")
split = 'train'  # 'train', 'val', 'test' を変更可能

# パスの確認（デバッグ用）
print("=" * 80)
print("パス確認")
print("=" * 80)
print(f"ソースディレクトリ: {source}")
source_path = Path(source)
print(f"  存在確認: {source_path.exists()}")
if source_path.exists():
    json_files = list(source_path.rglob('kaggle_data_*.json'))
    print(f"  見つかったJSONファイル数: {len(json_files)}")
    if len(json_files) > 0:
        print(f"  例: {json_files[0]}")
print(f"出力先ディレクトリ: {target}")
target_path = Path(target)
print(f"  存在確認: {target_path.exists()}")
print()

sys.argv = [
    'kaggle_labels.py',
    '--source', source,
    '--target', target,
    '--split', split
]

print("=" * 80)
print("前処理スクリプトを実行します")
print("=" * 80)
print(f"ソース: {source}")
print(f"出力先: {target}")
print(f"分割: {split}")
print()

# 実行
main()

In [ ]:
## 結果の確認（必要最低限）

生成されたラベルファイルの確認と統計情報の表示

### 生成されたラベルファイルの確認

In [ ]:
# 生成されたラベルファイルの確認
import sys
from pathlib import Path

# パスを追加（リポジトリのpython/dataディレクトリ）
repo_path = Path('/content/last_assignment_progzissen')
sys.path.insert(0, str(repo_path / 'python' / 'data'))

from config import CLASSES

out_root = os.environ.get("OUT_ROOT", "/content/drive/MyDrive/outputs")
split = 'train'  # Cell 3で設定した値と同じにする
labels_dir = Path(out_root) / "processed" / split / "labels"

label_files = list(labels_dir.glob('*.txt'))
if len(label_files) > 0:
    print(f"生成されたラベルファイル数: {len(label_files)}")
    print(f"\n最初のラベルファイルの例:")
    print(f"ファイル名: {label_files[0].name}")
    print("\n内容（最初の10行）:")
    with open(label_files[0], 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines[:10], 1):
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                class_name = CLASSES[class_id] if class_id < len(CLASSES) else 'unknown'
                print(f"  {i}. クラス: {class_name} (ID: {class_id}), "
                      f"bbox: ({parts[1]}, {parts[2]}, {parts[3]}, {parts[4]})")
        if len(lines) > 10:
            print(f"  ... 他 {len(lines) - 10} 行")
else:
    print("⚠️ ラベルファイルが見つかりません")
    print(f"確認: {labels_dir}")

### 統計情報の確認

In [ ]:
# クラス別統計を表示（生成されたラベルファイルから集計）
from collections import defaultdict

stats = defaultdict(int)
total_instances = 0

for label_file in label_files:
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                if 0 <= class_id < len(CLASSES):
                    stats[CLASSES[class_id]] += 1
                    total_instances += 1

print("クラス別統計:")
print("-" * 60)
for cls in CLASSES:
    count = stats[cls]
    print(f"  {cls:>3}: {count:>8} インスタンス")
print("-" * 60)
print(f"総インスタンス数: {total_instances}")
print(f"\n出力先: {labels_dir}")

### 統計情報をDataFrameで表示（見やすくするため）

In [ ]:
import pandas as pd

if total_instances > 0:
    df_stats = pd.DataFrame([
        {
            'クラス': cls,
            'インスタンス数': stats[cls],
            '割合 (%)': f"{stats[cls]/total_instances*100:.2f}"
        }
        for cls in CLASSES
    ])
    df_stats = df_stats.sort_values('インスタンス数', ascending=False)
    
    print("\nクラス別統計（降順）:")
    print(df_stats.to_string(index=False))
else:
    print("統計情報がありません")